In [17]:
import os
import json
import random
from enum import Enum
from typing import List, Tuple, Callable

BASE_PATH = os.path.dirname(os.getcwd())
ITEMS_PATH = f"{BASE_PATH}/data/churrasco.json"

In [53]:
class ItemCategory(Enum):
    CARNE_BOVINA = 1
    CARNE_SUINA = 2
    CARNE_AVES = 3
    OUTROS = 4


class Item:
    def __init__(self, name: str, batch_price: float,
        batch_quantity: float | None, increment: float,
        batch_kg: float, category: ItemCategory
    ) -> None:
        self.name = name
        self.category = category
        self.batch_kg = batch_kg
        self.increment = increment
        self.batch_price = batch_price
        self.batch_quantity = batch_quantity


class Barbecue:
    def __init__(self, money_limit: float) -> None:
        self.items: List[Item] = []
        self.mass_limit: float = None
        self.money_limit = money_limit
        self.objetive_compose: List[Tuple[Callable[[List[float], float]], float]] = []

    def set_objective_compose(self, compose: List[Tuple[Callable[[List[float]], float], float]]) -> None:
        self.objetive_compose = compose

    def random_solution(self) -> List[float]:
        while True:
            solution: List[float] = []
            for i in range(0, len(self.items)):
                multiply = random.randint(0, 10)
                quantity = self.items[i].increment * multiply
                solution.append(quantity)

            if self.__valid_solution(solution):
                break

        return solution

    def near_solution(self) -> List[List[float]]:
        pass

    def calculate_fitness(self, solution: List[float]) -> float:
        pass

    def calculate_spent(self, solution: List[float]) -> float:
        spent = 0
        for i in range(0, len(solution)):
            spent = spent + self.items[i].batch_price * solution[i]
        return spent / self.money_limit
    
    def calculate_mass(self, solution: List[float]) -> float:
        mass = 0
        for i in range(0, len(solution)):
            mass = mass + self.items[i].batch_kg * solution[i]
        return mass / self.mass_limit
    
    def calculate_diversity(self, solution: List[float]) -> float:
        # Encontra a massa de cada item
        mass = []
        for i in range(0, len(solution)):
            mass.append(self.items[i].batch_kg * solution[i])
        
        # Calcula a fração mássica de cada item
        mass_total = sum(mass)
        mass_mean = mass_total / len(mass)
        frac = [m / mass_mean for m in mass]

        # Define a diversidade como o mínimo entre da lista de
        # razão entre a fração real e a fração igualitária
        equal_frac = 1 / len(solution)
        frac = [f / equal_frac for f in frac]
        return min(frac)

    def process_json(self, data: dict) -> None:
        # Carrega itens do json
        items: List[Item] = []
        for item in data.keys():
            items.append(Item(
                item,
                data[item]["Preço/Lote"],
                data[item]["Quantidade/Lote"],
                data[item]["Incremento"],
                data[item]["Massa/Lote"],
                self.__format_category(data[item]["Categoria"])
            ))
        self.items = items

        # Calcula a massa máxima que pode ser obtida
        max_mass = 0
        for item in self.items:
            spent, multiply = 0, 0
            while spent < self.money_limit:
                spent = spent + item.increment * item.batch_price
                multiply = multiply + item.increment

            multiply = multiply - item.increment
            item_mass = multiply * item.batch_kg
            if item_mass > max_mass:
                max_mass = item_mass

        self.mass_limit = max_mass

    def __valid_solution(self, solution: List[float]) -> bool:
        spent = 0
        for i in range(0, len(solution)):
            spent = spent + self.items[i].batch_price * solution[i]

        if spent > self.money_limit:
            return False
        else:
            return True

    def __format_category(self, category: str) -> ItemCategory:
        if category == "Carne Bovina":
            return ItemCategory.CARNE_BOVINA
        elif category == "Carne Suína":
            return ItemCategory.CARNE_SUINA
        elif category == "Carne de Aves":
            return ItemCategory.CARNE_AVES
        else:
            return ItemCategory.OUTROS

In [80]:
with open(ITEMS_PATH) as file:
    data = json.load(file)

barbecue = Barbecue(money_limit=300)
barbecue.process_json(data)

solution = barbecue.random_solution()
print(solution)

fitness_spent = barbecue.calculate_spent(solution)
print(f"Fitness de gasto: {fitness_spent}")

fitness_mass = barbecue.calculate_mass(solution)
print(f"Fitness de massa: {fitness_mass}")

fitness_diversity = barbecue.calculate_diversity(solution)
print(f"Fitness de diversidade: {fitness_diversity}")

[1.0, 0.5, 0.1, 0.0, 0.0, 5]
Fitness de gasto: 0.46878000000000003
Fitness de massa: 0.38888888888888884
Fitness de diversidade: 0.0


In [41]:
barbecue.mass_limit

14.4